# Clustering Zooniverse Marks to count Iguanas
Count all the iguanas in the images by clustering the marks from the zooniverse volunteers.
This does not compare the results to the gold standard and requires only a single file, the flattened zooniverse data export.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("./")
sys.path.append("./zooniverse")

## Intro
### Retrieve a Classification report from Zooniverse
This notebook is used to cluster the marks from the zooniverse volunteers to count the iguanas in the images saved in results/<phase_tag>/flat_dataset_filtered_Iguanas 3rd launch.csv

Used Methods are:

### DBSCAN 
It does not require the number of clusters to be specified. It is used here because, but has min_samples and eps as hyperparameters which need to be found. [Link](https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html)
For finding eps and min_sample a simple **grid search** is used.
Additionally, DBSCAN not assume a specific shape for the clusters (K-means assumes clusters are gaussian in shape) even though we should assume that points around an iguana is gaussian shaped.

### HDBSCAN
It is an extension of DBSCAN which is more robust to hyperparameter settings as it finds epsilon and min_samples automatically. [Link](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html)

## Load the data

In [2]:
from pathlib import Path

import pandas as pd
from zooniverse.analysis import get_annotation_count_stats
from zooniverse.utils.filters import filter_df_user_threshold



# Phase Selection
phase_tag = "Iguanas 1st launch"
# phase_tag = "Iguanas 2nd launch"
# phase_tag = "Iguanas 3rd launch"


debug = False # debugging with a smaller dataset
plot_diagrams = False # plot the diagrams to disk for the clustering methods
show_plots = False # show the plots in the notebook

user_threshold = None # None or a number, filter records which have less than these user interactions.
use_gold_standard_subset = None # Use no filtering


## Input Path of all the data
input_path =Path("/Users/christian/data/zooniverse")
# input_path = Path("results/")
# Location for the analysis Results
output_path = Path(input_path.joinpath(f"2024_04_24_analysis").joinpath(phase_tag))
output_path.mkdir(exist_ok=True, parents=True)
reprocess = False # if True, the raw classification data is reprocessed. If False, the data is loaded from disk

# Location for plots
# output_plot_path = output_path.joinpath("plots")
# output_plot_path.mkdir(parents=True, exist_ok=True)
output_plot_path = None # do not plot

## Look into the config
This Config points to all files necessary for the analysis + the result files

In [3]:
from zooniverse.config import get_config

config = get_config(phase_tag=phase_tag, input_path=input_path, output_path=output_path)
config

{'annotations_source': PosixPath('/Users/christian/data/zooniverse/XYZ/2023-10-15/iguanas-from-above-classifications.csv'),
 'goldstandard_data': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/expert-GS-1stphase.csv'),
 'gold_standard_image_subset': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/1-T2-GS-results-5th-0s.csv'),
 'image_source': PosixPath('/Users/christian/data/zooniverse/Images/Zooniverse_Goldstandard_images/1st launch'),
 'yes_no_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_24_analysis/Iguanas 1st launch/yes_no_dataset_Iguanas 1st launch.csv'),
 'flat_dataset': PosixPath('/Users/christian/data/zooniverse/2024_04_24_analysis/Iguanas 1st launch/flat_dataset_Iguanas 1st launch.csv'),
 'flat_panoptes_points': PosixPath('/Users/christian/data/zooniverse/2024_04_24_analysis/Iguanas 1st launch/flat_panoptes_points_Iguanas 1st launch.csv'),
 'panoptes_question': PosixPath('/Users/christian/dat

In [4]:
config["flat_panoptes_points"]

PosixPath('/Users/christian/data/zooniverse/2024_04_24_analysis/Iguanas 1st launch/flat_panoptes_points_Iguanas 1st launch.csv')

## Look at the data


In [5]:

df_flat_panoptes_points = pd.read_csv(config["flat_panoptes_points"])
df_flat_panoptes_points["phase"] = phase_tag

df_flat_panoptes_points

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y,phase
0,283007416,77dd9c009b5200eea39788f7f15862f0,003abe9e6bb90e03c50377a33f1137a5,14370,134.236,T2,2020-10-26 09:28:46 UTC,47974351,SMF02-1-2-2_570.jpg,186,310,Iguanas 1st launch
1,271011060,fcbbe951169dc1df0b1f803a199f0995,004eb4442ae5c85fcb6b5c78ed902f41,14370,134.236,T2,2020-08-30 16:22:33 UTC,47978494,SFP01_06-2_59.jpg,272,265,Iguanas 1st launch
2,286621582,59e4feed8a3a84fb6bfdc5f29a1713b4,006c16c227f4152ffff80d195fca443b,14370,134.236,T2,2020-11-12 11:08:10 UTC,47980877,SRL04-1_348.jpg,358,194,Iguanas 1st launch
3,286621582,59e4feed8a3a84fb6bfdc5f29a1713b4,006c16c227f4152ffff80d195fca443b,14370,134.236,T2,2020-11-12 11:08:10 UTC,47980877,SRL04-1_348.jpg,485,106,Iguanas 1st launch
4,286621582,59e4feed8a3a84fb6bfdc5f29a1713b4,006c16c227f4152ffff80d195fca443b,14370,134.236,T2,2020-11-12 11:08:10 UTC,47980877,SRL04-1_348.jpg,458,205,Iguanas 1st launch
...,...,...,...,...,...,...,...,...,...,...,...,...
34540,269965841,46c1b35e9e7a99fe391c096e87a43677,NaN,14370,134.236,T2,2020-08-24 15:39:55 UTC,48034453,SRBS03-4_40.jpg,484,550,Iguanas 1st launch
34541,278911094,ec6e6d9157dc272dae21504deaa63687,NaN,14370,134.236,T2,2020-10-08 15:07:18 UTC,48034455,SRBS03-4_43.jpg,742,531,Iguanas 1st launch
34542,280834231,28e1c957b094499b9178b64aadab669e,NaN,14370,134.236,T2,2020-10-16 07:02:12 UTC,48034455,SRBS03-4_43.jpg,1412,1483,Iguanas 1st launch
34543,283374116,b35f4fa3408911f68c9c80ba68b8611f,NaN,14370,134.236,T2,2020-10-27 21:11:40 UTC,48034465,SRBS03-4_61.jpg,1458,1283,Iguanas 1st launch


### Filter User if necessary and Marks


In [6]:
# Use the panoptes Extraction instead of the custom extraction
df_merged_dataset = df_flat_panoptes_points

### Are there anonymous users in the data?
There should be some because anonymous users are kept.

In [7]:
df_merged_dataset[df_merged_dataset.user_id.isnull().values]

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y,phase
24972,269308827,52a54f2ec669d49acba3e42c3cb548cf,NaN,14370,134.236,T2,2020-08-21 03:24:16 UTC,47967503,SFB01-4-1_129.jpg,305,641,Iguanas 1st launch
24973,278487569,b827814c24463951126d00d3fff41d71,NaN,14370,134.236,T2,2020-10-06 20:04:13 UTC,47967515,SFB01-4-1_142.jpg,44,104,Iguanas 1st launch
24974,286283988,d3766176f1c7d46b672d58260c4d4621,NaN,14370,134.236,T2,2020-11-10 19:54:38 UTC,47967515,SFB01-4-1_142.jpg,474,242,Iguanas 1st launch
24975,280978577,43a478aa875a958fc1eee55eb89139f2,NaN,14370,134.236,T2,2020-10-16 19:56:04 UTC,47967518,SFB01-4-1_145.jpg,838,47,Iguanas 1st launch
24976,280978577,43a478aa875a958fc1eee55eb89139f2,NaN,14370,134.236,T2,2020-10-16 19:56:04 UTC,47967518,SFB01-4-1_145.jpg,727,65,Iguanas 1st launch
...,...,...,...,...,...,...,...,...,...,...,...,...
34540,269965841,46c1b35e9e7a99fe391c096e87a43677,NaN,14370,134.236,T2,2020-08-24 15:39:55 UTC,48034453,SRBS03-4_40.jpg,484,550,Iguanas 1st launch
34541,278911094,ec6e6d9157dc272dae21504deaa63687,NaN,14370,134.236,T2,2020-10-08 15:07:18 UTC,48034455,SRBS03-4_43.jpg,742,531,Iguanas 1st launch
34542,280834231,28e1c957b094499b9178b64aadab669e,NaN,14370,134.236,T2,2020-10-16 07:02:12 UTC,48034455,SRBS03-4_43.jpg,1412,1483,Iguanas 1st launch
34543,283374116,b35f4fa3408911f68c9c80ba68b8611f,NaN,14370,134.236,T2,2020-10-27 21:11:40 UTC,48034465,SRBS03-4_61.jpg,1458,1283,Iguanas 1st launch


In [8]:
# Amount of images
df_merged_dataset["subject_id"].nunique()

7854

In [9]:
## After filtering there
df_merged_dataset

,classification_id,user_name,user_id,workflow_id,workflow_version,task,created_at,subject_id,image_name,x,y,phase
0,283007416,77dd9c009b5200eea39788f7f15862f0,003abe9e6bb90e03c50377a33f1137a5,14370,134.236,T2,2020-10-26 09:28:46 UTC,47974351,SMF02-1-2-2_570.jpg,186,310,Iguanas 1st launch
1,271011060,fcbbe951169dc1df0b1f803a199f0995,004eb4442ae5c85fcb6b5c78ed902f41,14370,134.236,T2,2020-08-30 16:22:33 UTC,47978494,SFP01_06-2_59.jpg,272,265,Iguanas 1st launch
2,286621582,59e4feed8a3a84fb6bfdc5f29a1713b4,006c16c227f4152ffff80d195fca443b,14370,134.236,T2,2020-11-12 11:08:10 UTC,47980877,SRL04-1_348.jpg,358,194,Iguanas 1st launch
3,286621582,59e4feed8a3a84fb6bfdc5f29a1713b4,006c16c227f4152ffff80d195fca443b,14370,134.236,T2,2020-11-12 11:08:10 UTC,47980877,SRL04-1_348.jpg,485,106,Iguanas 1st launch
4,286621582,59e4feed8a3a84fb6bfdc5f29a1713b4,006c16c227f4152ffff80d195fca443b,14370,134.236,T2,2020-11-12 11:08:10 UTC,47980877,SRL04-1_348.jpg,458,205,Iguanas 1st launch
...,...,...,...,...,...,...,...,...,...,...,...,...
34540,269965841,46c1b35e9e7a99fe391c096e87a43677,NaN,14370,134.236,T2,2020-08-24 15:39:55 UTC,48034453,SRBS03-4_40.jpg,484,550,Iguanas 1st launch
34541,278911094,ec6e6d9157dc272dae21504deaa63687,NaN,14370,134.236,T2,2020-10-08 15:07:18 UTC,48034455,SRBS03-4_43.jpg,742,531,Iguanas 1st launch
34542,280834231,28e1c957b094499b9178b64aadab669e,NaN,14370,134.236,T2,2020-10-16 07:02:12 UTC,48034455,SRBS03-4_43.jpg,1412,1483,Iguanas 1st launch
34543,283374116,b35f4fa3408911f68c9c80ba68b8611f,NaN,14370,134.236,T2,2020-10-27 21:11:40 UTC,48034465,SRBS03-4_61.jpg,1458,1283,Iguanas 1st launch


In [10]:
# how many marks per user
df_merged_dataset[["user_id", "user_name", "x"]].groupby("user_name").count()

,user_id,x
user_name,,
0009862a245b7feba6c040fd1ef70b6b,0,3
001a7e5e989807349d0dc5e90eaebf83,1,1
0020d11acf216aee92a2f0d65d40e6b7,0,1
007f9c0b88de8a0716d5438948a196a9,1,1
00881a524994cde8bb34f5c062ff7c28,30,30
...,...,...
ffa31db924ebf1353aaac21d4c6a3ff1,0,6
ffd06a149e3961fd44f45c2f57fac9f8,10,10
ffd25020cf2308578be84d51a5293adf,6,6


## Clustering

### Basic Statics like mean, median, mode

In [11]:
from zooniverse.analysis import get_mark_overview

basic_stats = []

for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)

    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=df_image_name.iloc[0]["image_name"], 
                                                         subject_id=df_image_name.iloc[0]["subject_id"])

    ### basic statistics like mean, median, mode
    basic_stats.append(annotations_count_stats)
    

df_basic_stats = pd.DataFrame(basic_stats)    
df_comparison = df_basic_stats


In [12]:
df_basic_stats

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
0,SFB01-3_08.jpg,47967468,1.0,1.0,1,1,[1],1.0,2,2,"[1, 1]"
1,SFB01-4-1_103.jpg,47967478,1.0,1.0,1,1,[1],1.0,1,1,[1]
2,SFB01-4-1_108.jpg,47967483,1.0,1.0,1,1,[1],1.0,1,1,[1]
3,SFB01-4-1_115.jpg,47967489,1.0,1.0,1,1,[1],1.0,1,1,[1]
4,SFB01-4-1_129.jpg,47967503,1.0,1.0,1,1,[1],1.0,1,1,[1]
...,...,...,...,...,...,...,...,...,...,...,...
7849,SRBS03-4_61.jpg,48034465,1.0,1.0,1,1,[1],1.0,2,2,"[1, 1]"
7850,SRBS03-4_62.jpg,48034466,5.0,5.0,5,5,[5],5.0,1,5,[5]
7851,SRBS03-4_63.jpg,48034467,1.0,1.0,1,1,[1],1.0,1,1,[1]
7852,SRBS03-4_68.jpg,48034470,1.0,1.0,1,1,[1],1.0,1,1,[1]


In [13]:
# There might be records with too few annotations if they were not removed before
df_comparison[(df_comparison.sum_annotations_count < 5)].sort_values(by="users", ascending=False)

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
6813,SRPB05-25-2_134.jpg,47990222,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
2982,SFP01_06-1-3_150.jpg,47977314,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
483,SFB02-3-1_45.jpg,47969039,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
6592,SRPC01-2-2_116.jpg,47989312,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
3504,SRCBN01-3-1_55.jpg,47978909,1.0,1.0,1,1,[1],1.0,4,4,"[1, 1, 1, 1]"
...,...,...,...,...,...,...,...,...,...,...,...
2911,SFP01_06-1-2_300.jpg,47977122,2.0,2.0,2,2,[2],2.0,1,2,[2]
2908,SRCBN01-2-2_104.jpg,47977117,1.0,1.0,1,1,[1],1.0,1,1,[1]
2907,SFP01_06-1-2_288.jpg,47977108,1.0,1.0,1,1,[1],1.0,1,1,[1]
2906,SRCB01-3_141.jpg,47977106,1.0,1.0,1,1,[1],1.0,1,1,[1]


In [14]:
# Fill NaN values with 0
df_comparison.fillna(0, inplace=True)


### DBSCAN clustering and take the variant with the best silouette score for each image
Grid search for the best silouette score among the permutations of eps and min_samples values.

In [13]:
## sort by silouette score sorting + dbscan_count
# from zooniverse.analysis import compare_dbscan
# 
# eps_variants = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
# min_samples_variants = [3, 5, 8, 10]
# if debug:
#     eps_variants = [0.3]
#     min_samples_variants = [3]
# params = [(eps, min_samples) for eps in eps_variants for min_samples in min_samples_variants]
# 
# db_scan_results = {}
# db_scan_best_results = []
# for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
#     # check for at least 5 marks
#     if df_image_name.shape[0] >= 5:
#         dbscan_localization = compare_dbscan(
#             params=params,
#             df_flat=df_image_name,
#             output_plot_path=output_plot_path,
#             plot=show_plots
#         )
#     
#         db_scan_results[image_name] = pd.DataFrame(dbscan_localization)
#         
#         # DBSCAN tends to classify all points as noise if min_samples is too high. Often only a single user marked an iguana.
#         # Sillouette Scoring needs a minimum of 2 clusters
#         # if there are points in decent radius they will belong to a cluster
#         if pd.DataFrame(dbscan_localization).dbscan_count.max() == 1:
#             db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values("dbscan_count", ascending=False).iloc[0])
#             # If two or more cluster seem to exists take ones with the best Silouette score
#         else:  
#             # take the best result by silouette score if there are more clusters then 1
#             db_scan_best_results.append(pd.DataFrame(dbscan_localization).sort_values(["dbscan_silouette_score", "dbscan_count"], ascending=[False, False]).iloc[0])
#     
# df_dbscan_localization = pd.concat([*db_scan_results.values()])
# df_scan_best_results = pd.DataFrame(db_scan_best_results)


KeyboardInterrupt: 

In [14]:
# df_scan_best_results

NameError: name 'df_scan_best_results' is not defined

Here it can be seen why the silouette score is difficult because it is often undefined.

In [ ]:
## save the combinations of parameters, which maximized the silouette score.
# df_dbscan_localization.to_csv(config["dbscan_hyperparam_grid"])


In [ ]:
# df_scan_best_results.rename(columns={"dbscan_count": "dbscan_count_sil"}, inplace=True)
# 
# 
# df_comparison = df_comparison.merge(df_scan_best_results.drop(["image_name"], axis=1), on='subject_id', how='left')

In [15]:
# drop columns which are not interesting here
df_comparison = df_comparison.drop(["dbscan_noise", "dbscan_silouette_score", "eps", "min_samples", "dbscan_BIC_score", "with_noise", "bic_avg"], axis=1, errors="ignore")
df_comparison

,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count
0,SFB01-3_08.jpg,47967468,1.0,1.0,1,1,[1],1.0,2,2,"[1, 1]"
1,SFB01-4-1_103.jpg,47967478,1.0,1.0,1,1,[1],1.0,1,1,[1]
2,SFB01-4-1_108.jpg,47967483,1.0,1.0,1,1,[1],1.0,1,1,[1]
3,SFB01-4-1_115.jpg,47967489,1.0,1.0,1,1,[1],1.0,1,1,[1]
4,SFB01-4-1_129.jpg,47967503,1.0,1.0,1,1,[1],1.0,1,1,[1]
...,...,...,...,...,...,...,...,...,...,...,...
7849,SRBS03-4_61.jpg,48034465,1.0,1.0,1,1,[1],1.0,2,2,"[1, 1]"
7850,SRBS03-4_62.jpg,48034466,5.0,5.0,5,5,[5],5.0,1,5,[5]
7851,SRBS03-4_63.jpg,48034467,1.0,1.0,1,1,[1],1.0,1,1,[1]
7852,SRBS03-4_68.jpg,48034470,1.0,1.0,1,1,[1],1.0,1,1,[1]


### HDBSCAN clustering for each image


In [16]:
from zooniverse.analysis import hdbscan

hdbscan_values = []

eps_variants = [0.0] # 0 is the default
min_cluster_sizes = [5] # 5 is the default


for image_name, df_image_name in df_merged_dataset.groupby("subject_id"):
    annotations_count = get_mark_overview(df_image_name)
    
    image_name=df_image_name.iloc[0]["image_name"]
    subject_id=df_image_name.iloc[0]["subject_id"]
    
    annotations_count_stats = get_annotation_count_stats(annotations_count=annotations_count,
                                                         image_name=image_name, 
                                                         subject_id=subject_id)
    
    # if less than min_cluster_sizes points are available clustering makes no sense
    if df_image_name.shape[0] >= 5: # If num_samples is 5 for the min_cluster_size is 5 there is no point in passing data with less than 5 samples
        params = [(eps, min_cluster_size, max_cluster_size) 
                    for eps in eps_variants
                    for min_cluster_size in min_cluster_sizes
                    for max_cluster_size in [None]
              ]

        df_hdbscan = hdbscan(df_marks=df_image_name[["x", "y"]],
                                     output_path=output_plot_path,
                                     plot=show_plots,
                                     show=show_plots,
                                     image_name=image_name,
                                     subject_id=subject_id,
                                     params=params)
        hdbscan_values.append(df_hdbscan)


df_hdbscan = pd.concat(hdbscan_values)



In [17]:
df_hdbscan.drop(["with_noise"], axis=1, inplace=True)
df_hdbscan

,image_name,subject_id,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,SFB01-4-1_145.jpg,47967518,1,0.0,5,None,1
0,SFB01-4-1_216.jpg,47967585,1,0.0,5,None,12
0,SFBay-1-1_102.jpg,47967586,1,0.0,5,None,4
0,SFBay-1-1_28.jpg,47967600,1,0.0,5,None,2
0,SFBay-1-1_59.jpg,47967612,1,0.0,5,None,14
...,...,...,...,...,...,...,...
0,SRBS03-2-2_91.jpg,48034370,1,0.0,5,None,11
0,SRBS03-4_25.jpg,48034447,1,0.0,5,None,6
0,SRBS03-4_40.jpg,48034453,1,0.0,5,None,9
0,SRBS03-4_62.jpg,48034466,1,0.0,5,None,0


In [ ]:
## Statistics for all images
df_comparison = df_comparison.merge(df_hdbscan.drop(["image_name"], axis=1), on=['subject_id'], how='left')
df_comparison.fillna(0, inplace=True)
df_comparison

In [23]:
df_panoptes_question = pd.read_csv(config["panoptes_question"])
df_panoptes_question

,subject_id,data.no,data.yes
0,47967468,18.0,2.0
1,47967469,19.0,0.0
2,47967470,20.0,0.0
3,47967471,20.0,0.0
4,47967472,18.0,0.0
...,...,...,...
24368,48034473,20.0,0.0
24369,48034474,18.0,0.0
24370,48034475,18.0,0.0
24371,48034476,18.0,0.0


In [24]:
df_comparison.to_csv(config["comparison_dataset"])
print(f"saved {config['comparison_dataset']}")

saved /Users/christian/data/zooniverse/2024_04_24_analysis/Iguanas 1st launch/Iguanas 1st launch_method_comparison.csv


## Join the Expert Dataset for comparison
This dataset contains total_counts found by experts. 

In [25]:
df_expert_count = pd.read_csv(config["goldstandard_data"], sep=";")
# check what is in there
df_expert_count[df_expert_count.image_name == "ESCG02-1_19.jpg"]

,subspecies,island,site_name,subject_group,image_name,subject_id,presence_absence,count_male-lek,count_male-no-lek,count_others,count_partial,count_total,quality,condition,comment


In [26]:
## join the gold standard data to the basic stats
df_comparison_expert = df_expert_count[["site_name", "subject_group", "image_name", "subject_id", "count_total"]].merge(df_comparison.drop(["image_name"], axis=1), on='subject_id', how='left')

columns_to_fill = ['median_count', 'mean_count', 'mode_min_count', 'mode_max_count',
       'mode_count', 'mode_count_avg', 'users', 'sum_annotations_count',
        'dbscan_count_sil', 'HDBSCAN_count']
df_comparison_expert[columns_to_fill] = df_comparison_expert[columns_to_fill].fillna(0)

df_comparison_expert.sort_values(by="median_count", ascending=False)

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
847,El Miedo,SFM1,SFM01-2-2-3_79.jpg,47970332,0,287.0,287.0,287.0,287.0,[287],287.0,1.0,287.0,[287],3.0,1.0,0.0,5.0,0.0,33.0
498,El Miedo,SFM1,SFM01-2-2-1_101.jpg,47969229,0,160.0,160.0,160.0,160.0,[160],160.0,1.0,160.0,[160],2.0,4.0,0.0,5.0,0.0,30.0
2390,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-1_101.jpg,47987552,0,48.0,48.0,48.0,48.0,[48],48.0,1.0,48.0,[48],2.0,1.0,0.0,5.0,0.0,34.0
849,El Miedo,SFM1,SFM01-2-2-3_81.jpg,47970336,0,25.0,25.0,25.0,25.0,[25],25.0,1.0,25.0,[25],2.0,1.0,0.0,5.0,0.0,20.0
260,El Miedo,SFM1,SFM01-1-2_304.jpg,47968437,0,23.0,23.0,23.0,23.0,[23],23.0,1.0,23.0,[23],2.0,1.0,0.0,5.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,Loberia,SRL1,SRL01-1-2_229.jpg,47979779,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1161,Loberia,SRL1,SRL01-1-2_226.jpg,47979769,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1160,Loberia,SRL1,SRL01-1-2_225.jpg,47979765,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1159,Loberia,SRL1,SRL01-1-2_224.jpg,47979761,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [27]:
print(f"df_comparison_expert: {config['comparison_dataset_expert']}")
df_comparison_expert.to_csv(config["comparison_dataset_expert"])

df_comparison_expert: /Users/christian/data/zooniverse/2024_04_24_analysis/Iguanas 1st launch/Iguanas 1st launch_method_comparison_expert.csv


In [28]:
df_comparison_yes_no = df_comparison_expert.merge(df_panoptes_question, on="subject_id", how="left")
df_comparison_yes_no

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,...,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points,data.no,data.yes
0,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,2,2.0,2.0,1.0,3.0,"[1, 2, 3]",...,6.0,"[1, 2, 3]",1.0,1.0,0.0,5.0,0.0,1.0,14.0,5.0
1,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,4,1.0,1.7,1.0,1.0,[1],...,17.0,"[1, 1, 1, 1, 1, 1, 1, 3, 3, 4]",3.0,1.0,0.0,5.0,0.0,12.0,7.0,12.0
2,El Miedo,SFM1,SFM01-2-2-2_270.jpg,47969760,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,19.0,0.0
3,El Miedo,SFM1,SFM01-2-2-2_289.jpg,47969815,0,1.0,1.0,1.0,1.0,[1],...,1.0,[1],0.0,0.0,0.0,0.0,0.0,0.0,15.0,2.0
4,El Miedo,SFM1,SFM01-2-2-2_291.jpg,47969821,0,1.0,1.0,1.0,1.0,[1],...,1.0,[1],0.0,0.0,0.0,0.0,0.0,0.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_95.jpg,47990261,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,19.0,0.0
2729,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_96.jpg,47990268,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,18.0,0.0
2730,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_97.jpg,47990280,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,16.0,0.0
2731,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_98.jpg,47990285,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0


In [29]:
df_comparison_yes_no.to_csv(config["comparison_dataset_yes_no"])
df_comparison_yes_no

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,...,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points,data.no,data.yes
0,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,2,2.0,2.0,1.0,3.0,"[1, 2, 3]",...,6.0,"[1, 2, 3]",1.0,1.0,0.0,5.0,0.0,1.0,14.0,5.0
1,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,4,1.0,1.7,1.0,1.0,[1],...,17.0,"[1, 1, 1, 1, 1, 1, 1, 3, 3, 4]",3.0,1.0,0.0,5.0,0.0,12.0,7.0,12.0
2,El Miedo,SFM1,SFM01-2-2-2_270.jpg,47969760,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,19.0,0.0
3,El Miedo,SFM1,SFM01-2-2-2_289.jpg,47969815,0,1.0,1.0,1.0,1.0,[1],...,1.0,[1],0.0,0.0,0.0,0.0,0.0,0.0,15.0,2.0
4,El Miedo,SFM1,SFM01-2-2-2_291.jpg,47969821,0,1.0,1.0,1.0,1.0,[1],...,1.0,[1],0.0,0.0,0.0,0.0,0.0,0.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_95.jpg,47990261,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,19.0,0.0
2729,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_96.jpg,47990268,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,18.0,0.0
2730,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_97.jpg,47990280,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,16.0,0.0
2731,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-3_98.jpg,47990285,0,0.0,0.0,0.0,0.0,0,...,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,21.0,0.0


# A look into the results

## The sum of the clustering
What is the sum of the methods

In [30]:
df_comparison_yes_no[df_comparison_yes_no["data.yes"] >= 5]

,site_name,subject_group,image_name,subject_id,count_total,median_count,mean_count,mode_min_count,mode_max_count,mode_count,...,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points,data.no,data.yes
0,El Miedo,SFM1,SFM01-2-2-2_282.jpg,47969795,2,2.0,2.00,1.0,3.0,"[1, 2, 3]",...,6.0,"[1, 2, 3]",1.0,1.0,0.0,5.0,0.0,1.0,14.0,5.0
1,El Miedo,SFM1,SFM01-2-2-1_344.jpg,47969531,4,1.0,1.70,1.0,1.0,[1],...,17.0,"[1, 1, 1, 1, 1, 1, 1, 3, 3, 4]",3.0,1.0,0.0,5.0,0.0,12.0,7.0,12.0
5,El Miedo,SFM1,SFM01-2-2-2_293.jpg,47969828,7,2.0,2.57,1.0,1.0,[1],...,36.0,"[1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 5, 8]",3.0,1.0,0.0,5.0,0.0,31.0,3.0,14.0
22,El Miedo,SFM1,SFM01-1-1_114.jpg,47967876,1,1.0,1.44,1.0,1.0,[1],...,13.0,"[1, 1, 1, 1, 1, 1, 1, 2, 4]",1.0,1.0,0.0,5.0,0.0,8.0,8.0,11.0
60,El Miedo,SFM1,SFM01-1-1_154.jpg,47967959,1,1.0,1.00,1.0,1.0,[1],...,11.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1.0,1.0,0.0,5.0,0.0,6.0,6.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-1_67.jpg,47987920,1,1.0,1.07,1.0,1.0,[1],...,15.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]",1.0,1.0,0.0,5.0,0.0,10.0,2.0,16.0
2506,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-2_143.jpg,47988474,4,4.0,3.89,4.0,4.0,[4],...,35.0,"[2, 3, 3, 4, 4, 4, 4, 5, 6]",4.0,4.0,0.0,5.0,0.0,2.0,11.0,9.0
2516,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-2_153.jpg,47988580,1,1.0,1.00,1.0,1.0,[1],...,10.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.0,1.0,0.0,5.0,0.0,5.0,10.0,10.0
2524,Playa Blanca (Punta Pitt),SRPB1,SRPB02-2-2_161.jpg,47988636,1,1.0,1.00,1.0,1.0,[1],...,10.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0.0,1.0,0.0,5.0,0.0,5.0,9.0,10.0


In [31]:
# Take only the subjects which have more or equal 5 yes votes.
df_method_sums = df_comparison_yes_no[df_comparison_yes_no["data.yes"] >= 5][["count_total", "median_count", "mean_count", "mode_min_count", "mode_max_count", "mode_count_avg", "mode_min_count", "mode_max_count", "dbscan_count_sil", "HDBSCAN_count"]].sum()
df_method_sums.to_csv(config["method_sums"])
df_method_sums

count_total         331.000000
median_count        235.500000
mean_count          271.410000
mode_min_count      176.000000
mode_max_count      243.000000
mode_count_avg      208.066667
mode_min_count      176.000000
mode_max_count      243.000000
dbscan_count_sil    254.000000
HDBSCAN_count       245.000000
dtype: float64

## Compare the numbers
The counts are only for images which were in the dataset after filtering.

### Sum of all the Methods

In [32]:
print(f"{config['method_sums'].name}")
df_method_sums = pd.read_csv(config["method_sums"])
df_method_sums

Iguanas 1st launch_method_sums.csv


,Unnamed: 0,0
0,count_total,331.000000
1,median_count,235.500000
2,mean_count,271.410000
3,mode_min_count,176.000000
4,mode_max_count,243.000000
5,mode_count_avg,208.066667
6,mode_min_count,176.000000
7,mode_max_count,243.000000
8,dbscan_count_sil,254.000000
9,HDBSCAN_count,245.000000


### Comparison per Image Level

In [33]:
print(f"load {config['comparison_dataset']}")
pd.read_csv(config["comparison_dataset"])

load /Users/christian/data/zooniverse/2024_04_24_analysis/Iguanas 1st launch/Iguanas 1st launch_method_comparison.csv


,Unnamed: 0,image_name,subject_id,median_count,mean_count,mode_min_count,mode_max_count,mode_count,mode_count_avg,users,sum_annotations_count,annotations_count,dbscan_count_sil,HDBSCAN_count,eps,min_cluster_size,max_cluster_size,noise_points
0,0,SFB01-3_08.jpg,47967468,1.0,1.0,1,1,[1],1.0,2,2,"[1, 1]",0.0,0.0,0.0,0.0,0,0.0
1,1,SFB01-4-1_103.jpg,47967478,1.0,1.0,1,1,[1],1.0,1,1,[1],0.0,0.0,0.0,0.0,0,0.0
2,2,SFB01-4-1_108.jpg,47967483,1.0,1.0,1,1,[1],1.0,1,1,[1],0.0,0.0,0.0,0.0,0,0.0
3,3,SFB01-4-1_115.jpg,47967489,1.0,1.0,1,1,[1],1.0,1,1,[1],0.0,0.0,0.0,0.0,0,0.0
4,4,SFB01-4-1_129.jpg,47967503,1.0,1.0,1,1,[1],1.0,1,1,[1],0.0,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7849,7849,SRBS03-4_61.jpg,48034465,1.0,1.0,1,1,[1],1.0,2,2,"[1, 1]",0.0,0.0,0.0,0.0,0,0.0
7850,7850,SRBS03-4_62.jpg,48034466,5.0,5.0,5,5,[5],5.0,1,5,[5],0.0,1.0,0.0,5.0,0,0.0
7851,7851,SRBS03-4_63.jpg,48034467,1.0,1.0,1,1,[1],1.0,1,1,[1],0.0,0.0,0.0,0.0,0,0.0
7852,7852,SRBS03-4_68.jpg,48034470,1.0,1.0,1,1,[1],1.0,1,1,[1],0.0,0.0,0.0,0.0,0,0.0
